In [ ]:
import pandas as pd
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
sw=stopwords.words('english')
ps=PorterStemmer()
import json 

In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
	SELECT ID, QGROUP, QPK, MAX_Q_PK, QOFFSET, MAX_QG_PK, QGOFFSET, MAX_FK_Form,QTEXT, IdText
	FROM vw_MetricsCompositeKeys WHERE PK_Form = '2020-A-SAOP'   
""" 
df = pd.read_sql(query,con=conn)   
df.to_excel(r'C:\_som\_src\_compile\saop\SAOP.xlsx','SAOP')  

In [ ]:
query = """ \
    SELECT  *FROM vwPickLists
    WHERE PK_PickListType IN (100,101,102,103,104,105,106,113,114,164,165,166)   
""" 
df_pt = pd.read_sql(query,con=conn)   
df_pt.to_csv(r'C:\_som\_src\_compile\saop\picks.csv', index=False)  
df_pt

In [ ]:
QGR = 3000
d = {} 
for qg in df.QGROUP.unique():
    QGR+=1
    d[str(qg)]= str(QGR)
d    

In [ ]:
QGOFFSET = -1
QPK = 24500 
for i,r in df.iterrows():      
    if QGOFFSET != r.QGOFFSET:
        QGOFFSET = r.QGOFFSET
        QPK += 25
    QPK+=1
    d[str(r.QPK)]= str(QPK)  

In [ ]:
with open("C:\_som\_src\_compile\saop\gapkeyval.json", "w") as f: 
    json.dump(d, f, indent=4)

In [ ]:
dfid = df
dfid.replace("", np.NaN, inplace=True) 
dfid.dropna(subset = ["IdText"], inplace=True)
dfid['QHASH'] = dfid.QTEXT.apply(encode)  

In [ ]:
did = {}
for i,r in df.iterrows(): 
    did[f'--\s{r.IdText}\s'] = '-- ' + r.IdText + ' IDT:{' + r.QHASH + '} '
with open("C:\_som\_src\_compile\saop\did.json", "w") as f: 
    json.dump(did, f, indent=4)
did    


In [ ]:
conn.close()

In [ ]:
t = """
Has the as Languages technologies platforms is your web don't we have web driver 
"""
def encode(t):
    t = re.sub('[^A-Za-z0-9]', ' ', t ) 
    t = ' '.join([w for w in t.split(' ') if w not in sw])
    #t = ' '.join([ps.stem(w) for w in t.split(' ')])
    t = t.upper()
    t = re.sub('[AEIOU\s]', '', t)  
    return t
encode(t)